In [5]:

def picas(n1, n2):
  P = 0
  for i in range(4):
    for j in range(4):
      if(i!=j):
        if(n1[i]==n2[j]):
          P += 1
  return P

def fijas(n1, n2):
  F = 0
  for i in range(4):
    if(n1[i]==n2[i]):
      F += 1
  return F

class Agent:
  def __init__(self):
    self.options = []
    self.last = None

  def ask(self):
    return self.options[len(self.options)//2]

  def init(self):
    self.option = []
    for i in range(10):
      for j in range(10):
        if(j!=i):
          for k in range(10):
            if(k!=i and k!=j):
              for l in range(10):
                if(l!=i and l!=j and l!=k):
                  self.options.append([i, j, k, l])
    return self.ask()

  def compute(self, percept ):
    if(percept == 'iniciar'):
      self.last = self.init()
    else:
      F = percept['fijas']
      if(F==4):
        return "Soy un dios en picas y fijas"
      P = percept['picas']
      opt = []
      for i in range(len(self.options)):
        P2 = picas(self.last, self.options[i])
        F2 = fijas(self.last, self.options[i])
        if(P==P2 and F==F2):
          opt.append(self.options[i])
      self.options = opt
      if(len(self.options)==0):
        return "Usuario tramposo no me diste la información correcta. Humano tenia que ser"
      self.last = self.ask()
    txt = ''
    for i in range(4):
      txt += str(self.last[i])
    return txt

# Programa principal
a = Agent()
a2 = Agent()
n = a.compute('iniciar')
while(True):
  print(n)
  f = int(input('Fijas?'))
  p = int(input('Picas?'))
  n = a.compute({'picas':p, 'fijas':f})

5012
7346
Usuario tramposo no me diste la información correcta. Humano tenia que ser


ValueError: invalid literal for int() with base 10: ''

In [12]:
class SmartAgent:
    def __init__(self):
        self.options = []
        self.last = None
        self.step = 0
        self.block_results = []  # resultados de los bloques de partición
        self.candidate_digits = set()  # dígitos candidatos identificados
    
    def init(self):
        # Generar todas las permutaciones de 4 dígitos únicos
        import itertools
        self.options = [''.join(p) for p in itertools.permutations('0123456789', 4)]
        self.step = 0
        self.block_results = []
        self.candidate_digits = set()
        self.last = "1234"  # Primer bloque: {1,2,3,4}
        return self.last
    
    def compute(self, percept):
        if percept == "iniciar":
            return self.init()
        
        f, p = percept['fijas'], percept['picas']
        total_matches = f + p
        
        # ¡VERIFICAR SIEMPRE SI YA GANAMOS!
        if f == 4:
            return f"¡GANÉ! El número secreto es: {self.last}"
        
        # Paso 1: Probar primer bloque {1,2,3,4}
        if self.step == 0:
            self.block_results.append(("1234", total_matches))
            self.step += 1
            self.last = "5678"  # Segundo bloque: {5,6,7,8}
            return self.last
        
        # Paso 2: Probar segundo bloque {5,6,7,8}
        elif self.step == 1:
            self.block_results.append(("5678", total_matches))
            self.step += 1
            
            # Analizar resultados de los bloques
            block1_total = self.block_results[0][1]  # coincidencias en {1,2,3,4}
            block2_total = self.block_results[1][1]  # coincidencias en {5,6,7,8}
            
            print(f"Bloque 1234: {block1_total} coincidencias")
            print(f"Bloque 5678: {block2_total} coincidencias")
            
            # Determinar dígitos candidatos
            if block1_total > 0:
                self.candidate_digits.update(['1','2','3','4'])
            if block2_total > 0:
                self.candidate_digits.update(['5','6','7','8'])
            
            # Si la suma es menor a 4, necesitamos dígitos de {0,9}
            if block1_total + block2_total < 4:
                self.candidate_digits.update(['0','9'])
            
            print(f"Dígitos candidatos identificados: {sorted(self.candidate_digits)}")
            
            # Reducir espacio de búsqueda a solo los dígitos candidatos
            self.options = [opt for opt in self.options 
                          if all(d in self.candidate_digits for d in opt)]
            
            print(f"Opciones restantes: {len(self.options)}")
            
            # Elegir siguiente intento basado en los dígitos candidatos
            if len(self.candidate_digits) == 4:
                # Si tenemos exactamente 4 dígitos, probar una permutación
                digits = sorted(self.candidate_digits)
                self.last = ''.join(digits)
            else:
                # Si tenemos más de 4, elegir los primeros 4
                digits = sorted(self.candidate_digits)[:4]
                self.last = ''.join(digits)
            
            return self.last
        
        # Paso 3+: Filtrar opciones según el último intento
        else:
            # Filtrar opciones que coincidan con las picas y fijas del último intento
            valid_options = []
            for opt in self.options:
                opt_picas = picas(opt, self.last)
                opt_fijas = fijas(opt, self.last)
                if opt_picas == p and opt_fijas == f:
                    valid_options.append(opt)
            
            self.options = valid_options
            
            if len(self.options) == 0:
                return "Error: No hay opciones válidas. Información inconsistente."
            
            if len(self.options) == 1:
                return f"¡Encontrado! El número secreto es: {self.options[0]}"
            
            # Elegir siguiente intento (estrategia simple: primera opción)
            self.last = self.options[0]
            return self.last


In [13]:
# Programa principal con SmartAgent
print("=== AGENTE INTELIGENTE CON ESTRATEGIA DE PARTICIONES ===")
print("Piensa en un número de 4 dígitos únicos (ej: 1234, 5678, 9012)")
print("El agente usará la estrategia de partición por bloques para encontrarlo eficientemente")
print()

smart_agent = SmartAgent()
n = smart_agent.compute('iniciar')
intentos = 0

while(True):
    intentos += 1
    print(f"\n--- Intento {intentos} ---")
    print(f"Agente propone: {n}")
    
    # Verificar si el agente ya ganó
    if "¡GANÉ!" in n or "¡Encontrado!" in n or "Error:" in n:
        print(n)
        break
        
    f = int(input('¿Cuántas fijas? '))
    p = int(input('¿Cuántas picas? '))
    
    # Verificar victoria inmediata
    if f == 4:
        print(f"¡El agente ganó en {intentos} intentos! El número secreto era: {n}")
        break
        
    n = smart_agent.compute({'picas':p, 'fijas':f})

=== AGENTE INTELIGENTE CON ESTRATEGIA DE PARTICIONES ===
Piensa en un número de 4 dígitos únicos (ej: 1234, 5678, 9012)
El agente usará la estrategia de partición por bloques para encontrarlo eficientemente


--- Intento 1 ---
Agente propone: 1234

--- Intento 2 ---
Agente propone: 5678
Bloque 1234: 1 coincidencias
Bloque 5678: 2 coincidencias
Dígitos candidatos identificados: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Opciones restantes: 5040

--- Intento 3 ---
Agente propone: 0123

--- Intento 4 ---
Agente propone: 1045

--- Intento 5 ---
Agente propone: 1056

--- Intento 6 ---
Agente propone: 1605

--- Intento 7 ---
Agente propone: ¡Encontrado! El número secreto es: 6015
¡Encontrado! El número secreto es: 6015


In [14]:
# Simulador automático para probar la estrategia
class Simulator:
    def __init__(self, secret_number):
        self.secret = secret_number
        self.attempts = 0
        
    def get_feedback(self, guess):
        """Simula la respuesta del jugador humano"""
        f = fijas(self.secret, guess)
        p = picas(self.secret, guess)
        self.attempts += 1
        return {'fijas': f, 'picas': p}
    
    def test_agent(self, agent):
        """Prueba el agente con el número secreto"""
        print(f"\n=== PROBANDO CON NÚMERO SECRETO: {self.secret} ===")
        
        # Reiniciar el agente
        guess = agent.compute('iniciar')
        print(f"Intento 1: {guess}")
        
        # Primer bloque (1234)
        feedback = self.get_feedback(guess)
        print(f"  → Fijas: {feedback['fijas']}, Picas: {feedback['picas']}")
        
        # Verificar si ya ganó en el primer intento
        if feedback['fijas'] == 4:
            print(f"  → ¡GANÓ EN EL PRIMER INTENTO! El número secreto era: {guess}")
            return self.attempts
        
        # Segundo bloque (5678)
        guess = agent.compute(feedback)
        print(f"Intento 2: {guess}")
        
        feedback = self.get_feedback(guess)
        print(f"  → Fijas: {feedback['fijas']}, Picas: {feedback['picas']}")
        
        # Verificar si ya ganó en el segundo intento
        if feedback['fijas'] == 4:
            print(f"  → ¡GANÓ EN EL SEGUNDO INTENTO! El número secreto era: {guess}")
            return self.attempts
        
        # Continuar hasta encontrar la solución
        while True:
            guess = agent.compute(feedback)
            print(f"Intento {self.attempts + 1}: {guess}")
            
            # Verificar si el agente dice que ganó
            if "¡GANÉ!" in guess or "¡Encontrado!" in guess:
                print(f"  → {guess}")
                break
                
            feedback = self.get_feedback(guess)
            print(f"  → Fijas: {feedback['fijas']}, Picas: {feedback['picas']}")
            
            # Verificar victoria
            if feedback['fijas'] == 4:
                print(f"  → ¡Solución encontrada en {self.attempts} intentos!")
                break
        
        return self.attempts

# Probar con diferentes casos
test_cases = ["1234", "5678", "9012", "1357", "2468", "7890"]
simulator = Simulator("1234")  # Se cambiará para cada caso

print("=== PRUEBAS AUTOMÁTICAS DE LA ESTRATEGIA ===")
for secret in test_cases:
    agent = SmartAgent()
    sim = Simulator(secret)
    attempts = sim.test_agent(agent)
    print(f"Total de intentos para {secret}: {attempts}")
    print("-" * 50)


=== PRUEBAS AUTOMÁTICAS DE LA ESTRATEGIA ===

=== PROBANDO CON NÚMERO SECRETO: 1234 ===
Intento 1: 1234
  → Fijas: 4, Picas: 0
  → ¡GANÓ EN EL PRIMER INTENTO! El número secreto era: 1234
Total de intentos para 1234: 1
--------------------------------------------------

=== PROBANDO CON NÚMERO SECRETO: 5678 ===
Intento 1: 1234
  → Fijas: 0, Picas: 0
Intento 2: 5678
  → Fijas: 4, Picas: 0
  → ¡GANÓ EN EL SEGUNDO INTENTO! El número secreto era: 5678
Total de intentos para 5678: 2
--------------------------------------------------

=== PROBANDO CON NÚMERO SECRETO: 9012 ===
Intento 1: 1234
  → Fijas: 0, Picas: 2
Intento 2: 5678
  → Fijas: 0, Picas: 0
Bloque 1234: 2 coincidencias
Bloque 5678: 0 coincidencias
Dígitos candidatos identificados: ['0', '1', '2', '3', '4', '9']
Opciones restantes: 360
Intento 3: 0123
  → Fijas: 0, Picas: 3
Intento 4: 1034
  → Fijas: 1, Picas: 1
Intento 5: 1209
  → Fijas: 0, Picas: 4
Intento 6: 2091
  → Fijas: 1, Picas: 3
Intento 7: ¡Encontrado! El número secreto e

In [16]:
# Versión optimizada del SmartAgent con estrategia de minimax
class OptimizedSmartAgent:
    def __init__(self):
        self.options = []
        self.last = None
        self.step = 0
        self.block_results = []
        self.candidate_digits = set()
    
    def init(self):
        import itertools
        self.options = [''.join(p) for p in itertools.permutations('0123456789', 4)]
        self.step = 0
        self.block_results = []
        self.candidate_digits = set()
        self.last = "1234"
        return self.last
    
    def get_best_guess(self):
        """Elige el mejor intento basado en la estrategia de minimax"""
        if len(self.options) == 1:
            return self.options[0]
        
        # Si tenemos pocas opciones, probar directamente
        if len(self.options) <= 6:
            return self.options[0]
        
        # Estrategia: elegir el intento que maximiza la información
        best_guess = None
        min_max_remaining = float('inf')
        
        # Probar con las primeras opciones como candidatos
        candidates = self.options[:min(20, len(self.options))]
        
        for guess in candidates:
            # Calcular cuántas opciones quedarían en el peor caso
            max_remaining = 0
            
            # Simular todas las posibles respuestas
            for f in range(5):  # 0 a 4 fijas
                for p in range(5):  # 0 a 4 picas
                    if f + p > 4:  # Imposible tener más de 4 coincidencias
                        continue
                    
                    # Contar opciones que darían esta respuesta
                    count = 0
                    for opt in self.options:
                        opt_picas = picas(opt, guess)
                        opt_fijas = fijas(opt, guess)
                        if opt_picas == p and opt_fijas == f:
                            count += 1
                    
                    max_remaining = max(max_remaining, count)
            
            # Elegir el intento que minimiza el peor caso
            if max_remaining < min_max_remaining:
                min_max_remaining = max_remaining
                best_guess = guess
        
        return best_guess if best_guess else self.options[0]
    
    def compute(self, percept):
        if percept == "iniciar":
            return self.init()
        
        f, p = percept['fijas'], percept['picas']
        total_matches = f + p
        
        # Paso 1: Probar primer bloque {1,2,3,4}
        if self.step == 0:
            self.block_results.append(("1234", total_matches))
            self.step += 1
            self.last = "5678"
            return self.last
        
        # Paso 2: Probar segundo bloque {5,6,7,8}
        elif self.step == 1:
            self.block_results.append(("5678", total_matches))
            self.step += 1
            
            # Analizar resultados de los bloques
            block1_total = self.block_results[0][1]
            block2_total = self.block_results[1][1]
            
            print(f"Bloque 1234: {block1_total} coincidencias")
            print(f"Bloque 5678: {block2_total} coincidencias")
            
            # Determinar dígitos candidatos
            if block1_total > 0:
                self.candidate_digits.update(['1','2','3','4'])
            if block2_total > 0:
                self.candidate_digits.update(['5','6','7','8'])
            
            if block1_total + block2_total < 4:
                self.candidate_digits.update(['0','9'])
            
            print(f"Dígitos candidatos: {sorted(self.candidate_digits)}")
            
            # Reducir espacio de búsqueda
            self.options = [opt for opt in self.options 
                          if all(d in self.candidate_digits for d in opt)]
            
            print(f"Opciones restantes: {len(self.options)}")
            
            # Usar estrategia optimizada para elegir siguiente intento
            self.last = self.get_best_guess()
            return self.last
        
        # Paso 3+: Filtrar y elegir mejor intento
        else:
            # Filtrar opciones
            valid_options = []
            for opt in self.options:
                opt_picas = picas(opt, self.last)
                opt_fijas = fijas(opt, self.last)
                if opt_picas == p and opt_fijas == f:
                    valid_options.append(opt)
            
            self.options = valid_options
            
            if len(self.options) == 0:
                return "Error: No hay opciones válidas."
            
            if len(self.options) == 1:
                return f"¡Encontrado! El número secreto es: {self.options[0]}"
            
            # Usar estrategia optimizada
            self.last = self.get_best_guess()
            return self.last

# Comparación de rendimiento
print("=== COMPARACIÓN DE ESTRATEGIAS ===")
print("SmartAgent básico vs OptimizedSmartAgent")
print("Probando con número secreto: 1357")
print()

# Probar agente básico
print("--- SmartAgent Básico ---")
agent_basic = SmartAgent()
sim_basic = Simulator("9012")
attempts_basic = sim_basic.test_agent(agent_basic)

print(f"\n--- OptimizedSmartAgent ---")
agent_opt = OptimizedSmartAgent()
sim_opt = Simulator("9012")
attempts_opt = sim_opt.test_agent(agent_opt)

print(f"\n=== RESULTADOS ===")
print(f"SmartAgent básico: {attempts_basic} intentos")
print(f"OptimizedSmartAgent: {attempts_opt} intentos")
print(f"Mejora: {attempts_basic - attempts_opt} intentos menos")


=== COMPARACIÓN DE ESTRATEGIAS ===
SmartAgent básico vs OptimizedSmartAgent
Probando con número secreto: 1357

--- SmartAgent Básico ---

=== PROBANDO CON NÚMERO SECRETO: 9012 ===
Intento 1: 1234
  → Fijas: 0, Picas: 2
Intento 2: 5678
  → Fijas: 0, Picas: 0
Bloque 1234: 2 coincidencias
Bloque 5678: 0 coincidencias
Dígitos candidatos identificados: ['0', '1', '2', '3', '4', '9']
Opciones restantes: 360
Intento 3: 0123
  → Fijas: 0, Picas: 3
Intento 4: 1034
  → Fijas: 1, Picas: 1
Intento 5: 1209
  → Fijas: 0, Picas: 4
Intento 6: 2091
  → Fijas: 1, Picas: 3
Intento 7: ¡Encontrado! El número secreto es: 9012
  → ¡Encontrado! El número secreto es: 9012

--- OptimizedSmartAgent ---

=== PROBANDO CON NÚMERO SECRETO: 9012 ===
Intento 1: 1234
  → Fijas: 0, Picas: 2
Intento 2: 5678
  → Fijas: 0, Picas: 0
Bloque 1234: 2 coincidencias
Bloque 5678: 0 coincidencias
Dígitos candidatos: ['0', '1', '2', '3', '4', '9']
Opciones restantes: 360
Intento 3: 0123
  → Fijas: 0, Picas: 3
Intento 4: 1234
  → Fi